In [5]:
import os
import re
from collections import defaultdict

REPO_DIR = r"C:\Users\harold.noble\Desktop\RIC\app\frontend\src"
SKIP_DIR = os.path.join(REPO_DIR, r"lib\components\icons")
OUTPUT_FILE = "dependency_analysis.txt"


def normalize_import_path(import_path, file_dir):
    """Normalize import paths to absolute paths within the repo."""
    if import_path.startswith(('./', '../')):
        abs_path = os.path.normpath(os.path.join(file_dir, import_path))
        return os.path.relpath(abs_path, REPO_DIR).replace('\\', '/')
    if import_path.startswith('/'):
        return import_path.lstrip('/')
    return import_path


def get_imports(content, file_dir):
    """Extract and normalize all import paths from the content."""
    import_patterns = [
        r'import.*from\s*[\'"](.+?)[\'"]',
        r'import\s*[\'"](.+?)[\'"]'
    ]
    normalized_imports = set()
    for pattern in import_patterns:
        for imp in re.findall(pattern, content):
            normalized_imports.add(normalize_import_path(imp, file_dir))
    return normalized_imports


def resolve_import_to_files(import_path, file_map):
    """Resolve an import path to actual file paths in the repo."""
    for ext in ['', '.ts', '.js', '.svelte', '/index.ts', '/index.js', '/index.svelte']:
        path = import_path + ext if not ext.startswith('/') else import_path + ext[1:]
        if path in file_map:
            return [path]
    return []


def process_file(file_path):
    """Process a file to extract its full content and imports."""
    file_dir = os.path.dirname(file_path)
    rel_file_path = os.path.relpath(file_path, REPO_DIR).replace('\\', '/')
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        imports = get_imports(content, file_dir)
        return rel_file_path, content, imports
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return rel_file_path, "", set()


def build_dependency_graph(file_info):
    """Build a graph of which files import which other files."""
    import_graph = defaultdict(set)  # file -> files that import it
    file_map = {os.path.relpath(path, REPO_DIR).replace('\\', '/'): path for path in file_info}

    for file_path, info in file_info.items():
        rel_path = os.path.relpath(file_path, REPO_DIR).replace('\\', '/')
        for import_path in info['imports']:
            resolved_files = resolve_import_to_files(import_path, file_map)
            for resolved in resolved_files:
                if resolved in file_map:
                    import_graph[file_map[resolved]].add(file_path)
    return import_graph, file_map


def analyze_dependencies(output_file=OUTPUT_FILE):
    """Analyze dependencies, including full file content."""
    # Collect file info
    file_info = {}
    for root, _, files in os.walk(REPO_DIR):
        if root.startswith(SKIP_DIR):
            continue
        for file in files:
            if file.endswith(('.svelte', '.ts', '.js')):
                file_path = os.path.join(root, file)
                rel_path, content, imports = process_file(file_path)
                file_info[file_path] = {'content': content, 'imports': imports}

    import_graph, file_map = build_dependency_graph(file_info)
    processed_files = set()
    group_num = 1

    with open(output_file, 'w', encoding='utf-8') as out:
        while len(processed_files) < len(file_info):
            unprocessed = {p: info for p, info in file_info.items() if p not in processed_files}
            if not unprocessed:
                break

            # Pick one file with the fewest imports
            base_path = min(unprocessed, key=lambda p: len(unprocessed[p]['imports']))
            base_info = file_info[base_path]
            rel_base_path = os.path.relpath(base_path, REPO_DIR).replace('\\', '/')

            out.write(f"## GROUP {group_num} - Starting with {rel_base_path}\n\n")
            out.write(f"### {rel_base_path}\n")
            out.write(f"{base_info['content']}\n\n")
            processed_files.add(base_path)

            # Include imported files
            for import_path in base_info['imports']:
                resolved = resolve_import_to_files(import_path, file_map)
                for res in resolved:
                    if res in file_map and file_map[res] not in processed_files:
                        imp_path = file_map[res]
                        rel_imp_path = os.path.relpath(imp_path, REPO_DIR).replace('\\', '/')
                        imp_info = file_info[imp_path]
                        out.write(f"### {rel_imp_path}\n")
                        out.write(f"{imp_info['content']}\n\n")
                        processed_files.add(imp_path)

            # Include importing files
            importers = import_graph.get(base_path, set())
            for imp_path in importers:
                if imp_path not in processed_files:
                    rel_imp_path = os.path.relpath(imp_path, REPO_DIR).replace('\\', '/')
                    imp_info = file_info[imp_path]
                    out.write(f"### {rel_imp_path}\n")
                    out.write(f"{imp_info['content']}\n\n")
                    processed_files.add(imp_path)

            out.write(f"{'='*80}\n\n")
            group_num += 1

        # Handle remaining isolated files
        remaining = {p for p in file_info if p not in processed_files}
        if remaining:
            out.write(f"## GROUP {group_num} - Remaining Isolated Files\n\n")
            for file_path in remaining:
                rel_path = os.path.relpath(file_path, REPO_DIR).replace('\\', '/')
                out.write(f"### {rel_path}\n")
                out.write(f"{file_info[file_path]['content']}\n\n")
                processed_files.add(file_path)
            out.write(f"{'='*80}\n\n")
            group_num += 1

    print(f"Processed {len(processed_files)} of {len(file_info)} files in {group_num-1} groups")
    if len(processed_files) < len(file_info):
        print(f"Warning: {len(file_info) - len(processed_files)} files were not processed!")


if __name__ == "__main__":
    analyze_dependencies()

Processed 289 of 289 files in 210 groups
